In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
import random
from collections import Counter
from collections import OrderedDict
# from ipynb.fs.full.dev_notebook import stringToQiskitSingleGate

from generator import generateCliffordCircuit

In [3]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [141]:
def split_circuit_by_barrier(circuit):
    qasm = circuit.qasm()
    prelude = []
    circuits = [[]]
    for line in qasm.splitlines():
        if any([line.startswith(t) for t in ['OPENQASM', 'include', 'qreg', 'creg']]):
            prelude.append(line)
        elif line.startswith('barrier'):
            circuits.append([])
        else:
            circuits[-1].append(line)
    circuits_with_prelude = [prelude+circuit for circuit in circuits]
    for circuit_with_prelude in circuits_with_prelude:
        yield qk.QuantumCircuit.from_qasm_str('\n'.join(circuit_with_prelude))

# for subcircuit in (split_circuit_by_barrier(qiskitCircuit)):
#     print(subcircuit)

In [4]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['Z', 'Z', 'I', 'H', 'X'], ['CNOT_T', 'I', 'I', 'CNOT_C', 'I'], ['X', 'I', 'H', 'I', 'Z'], ['I', 'I', 'I', 'CNOT_T', 'CNOT_C'], ['X', 'S', 'H', 'H', 'H']]
     ┌───┐ ░ ┌───┐ ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ Z ├─░─┤ X ├─░─┤ X ├─░───────░─┤ X ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░       ░ ├───┤
q_1: ┤ Z ├─░───┼───░─┤ I ├─░───────░─┤ S ├
     ├───┤ ░   │   ░ ├───┤ ░       ░ ├───┤
q_2: ┤ I ├─░───┼───░─┤ H ├─░───────░─┤ H ├
     ├───┤ ░   │   ░ ├───┤ ░ ┌───┐ ░ ├───┤
q_3: ┤ H ├─░───■───░─┤ I ├─░─┤ X ├─░─┤ H ├
     ├───┤ ░       ░ ├───┤ ░ └─┬─┘ ░ ├───┤
q_4: ┤ X ├─░───────░─┤ Z ├─░───■───░─┤ H ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


In [5]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    return pauli.evolve(cliff,frame="s")

In [6]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('YYXYX')

In [110]:
def params_list(operation_set, pauli_set=["X","Y","Z","I"]):
    '''This function generates a dictionary of parameters for the given operation set.
    Inputs: operation_set - A list of operations.
            pauli_set - A list of pauli operators.
    Outputs: params - A dictionary of parameters, with each set to 0.'''

    params = OrderedDict()
    for operation in operation_set:
        if operation == "CNOT_C" or operation == "CNOT_T":
            for pauli1 in pauli_set:
                for pauli2 in pauli_set:
                    pauli = pauli1+pauli2
                    params[("CNOT", pauli)] = 0
        else:
            for pauli in pauli_set:
                params[(operation, pauli)] = 0
    return params

print(params_list(singleGateSet))

OrderedDict([(('X', 'X'), 0), (('X', 'Y'), 0), (('X', 'Z'), 0), (('X', 'I'), 0), (('H', 'X'), 0), (('H', 'Y'), 0), (('H', 'Z'), 0), (('H', 'I'), 0), (('Z', 'X'), 0), (('Z', 'Y'), 0), (('Z', 'Z'), 0), (('Z', 'I'), 0), (('I', 'X'), 0), (('I', 'Y'), 0), (('I', 'Z'), 0), (('I', 'I'), 0), (('S', 'X'), 0), (('S', 'Y'), 0), (('S', 'Z'), 0), (('S', 'I'), 0)])


In [69]:
def remove_negatives(pauli):
    '''This function takes a pauli operator and removes the negative sign from it.'''

    if pauli.to_label()[0] == "-":
        return qk.quantum_info.Pauli(pauli.to_label()[1:])
    else:
        return pauli

In [111]:
def pauli_hist(circuit, pauli_initial, params_list=params_list(singleGateSet)):
    '''This function computes the histogram of pauli operators after the circuit has been applied.
    Inputs: circuit - A Qiskit Circuit, Clifford, or Gate object.
            pauli_initial - A Qiskit Pauli object.
            params_list - A dictionary of parameters for the circuit.
    Outputs: pauli_counts - A dictionary of pauli operators and their counts.'''

    pauli = pauli_initial

    for op_instructions in circuit.data:
        if op_instructions.operation.name.upper() == "ID" or op_instructions.operation.name.upper() == "I" or op_instructions.operation.name.upper() == "BARRIER":
            print("skipped ", op_instructions.operation.name.upper(), " operation")
            break
        elif op_instructions.operation.name.upper() == "CX" or op_instructions.operation.name.upper() == "CNOT":
            params_list[("CNOT_C", pauli.to_label()[0])] += 1
            params_list[("CNOT_T", pauli.to_label()[1])] += 1
            pauli = qk.quantum_info.Pauli(pauli.to_label()[1] + pauli.to_label()[0] + pauli.to_label()[2:])
        else:
            pauli = remove_negatives(pauli)
            params_list[(op_instructions.operation.name.upper(), pauli.to_label())] += 1
            pauli = Clifford_Permute(op_instructions.operation, pauli)

    return params_list

print(pauli_hist(qiskitCircuit, qk.quantum_info.Pauli('X')))

skipped  BARRIER  operation
OrderedDict([(('X', 'X'), 1), (('X', 'Y'), 0), (('X', 'Z'), 2), (('X', 'I'), 0), (('H', 'X'), 1), (('H', 'Y'), 0), (('H', 'Z'), 0), (('H', 'I'), 0), (('Z', 'X'), 0), (('Z', 'Y'), 0), (('Z', 'Z'), 1), (('Z', 'I'), 0), (('I', 'X'), 0), (('I', 'Y'), 0), (('I', 'Z'), 0), (('I', 'I'), 0), (('S', 'X'), 0), (('S', 'Y'), 0), (('S', 'Z'), 0), (('S', 'I'), 0)])


In [72]:
def generate_first_layer(length, pauli_set=["X","Y","Z","I"]):
    first_layer = ""
    for ele in np.random.choice(pauli_set, length):
        first_layer += ele
    return first_layer.upper()

print(generate_first_layer(5))

XIXXI


In [112]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

qiskitCircuit = transpileListToQiskitCircuit(generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet))
print(qiskitCircuit)
stringfirstlayer = generate_first_layer(qiskitCircuit.num_qubits)
first_row = pauli_hist(qiskitCircuit, qk.quantum_info.Pauli(stringfirstlayer))

     ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ S ├─░───────░─┤ I ├─░───■───░─┤ Z ├
     ├───┤ ░       ░ ├───┤ ░ ┌─┴─┐ ░ ├───┤
q_1: ┤ S ├─░───────░─┤ H ├─░─┤ X ├─░─┤ I ├
     ├───┤ ░       ░ ├───┤ ░ └───┘ ░ ├───┤
q_2: ┤ I ├─░───────░─┤ S ├─░───────░─┤ H ├
     ├───┤ ░ ┌───┐ ░ ├───┤ ░       ░ ├───┤
q_3: ┤ Z ├─░─┤ X ├─░─┤ H ├─░───────░─┤ X ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░       ░ ├───┤
q_4: ┤ Z ├─░───■───░─┤ I ├─░───────░─┤ Z ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


KeyError: ('S', 'XIZII')

In [149]:
def generate_row_A():
    '''This function generates a row of the circuit, which is a dictionary of pauli operators and their counts.'''
    # generate the random circuit
    random_circuit = transpileListToQiskitCircuit(generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet))
    print(random_circuit)

    # get the following information from circuit: 
    # 1) input pauli layer ex. 'XIZII', with an initial list of the paulis at each register that can be overwritten.
    # 2) empty dictionary of pauli operators and their counts ex. 
    # {('X', 'X'): 0, ('X', 'I'): 0, ('Z', 'Z'): 0, ('Z', 'I'): 0, ('I', 'X'): 0, ('I', 'I'): 0}
    # 3) all operations in the circuit ex. ['X', 'H', 'Z', 'I', 'S', 'CNOT_C', 'CNOT_T']
    input_pauli_layer = generate_first_layer(random_circuit.num_qubits)
    input_pauli = [char for char in input_pauli_layer]
    curr_row_dict = params_list(singleGateSet)
    all_operations = curr_row_dict.keys()
    print(curr_row_dict)

    # iterate each gate in each layer of the circuit 
    for layer in split_circuit_by_barrier(random_circuit):
        for gate in layer:

            # get the register of the current gate
            register = random_circuit.find_bit(gate.qubits[0]).index

            # get the gate's operator and the input pauli
            input_gate = gate.operation.name.upper()
            cur_pauli = input_pauli_layer[register]

            print("current operation is: ", register, input_gate, cur_pauli)

            # check if current operation is in dict already
            if (input_gate, cur_pauli) in all_operations:
                # if so, increment the count
                curr_row_dict[(input_gate, cur_pauli)] += 1
            # if it is an identity or a barrier, we don't care about it and it won't be in our dict so ignore
            elif input_gate == "ID" or input_gate == "I" or input_gate == "BARRIER":
                continue
            # if it is a CNOT, we need to increment the count of the control and target.
            # NOT IMPLEMENTED YET
            elif input_gate == "CX" or input_gate == "CNOT":
                print("skipped ", input_gate, " operation")
            else: 
                print("failed here with ", input_gate, cur_pauli)

    
    print(curr_row_dict.values())
    return curr_row_dict.values()
        

generate_row_A()

     ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ I ├─░───────░─┤ Z ├─░───■───░─┤ X ├
     ├───┤ ░ ┌───┐ ░ ├───┤ ░   │   ░ ├───┤
q_1: ┤ S ├─░─┤ X ├─░─┤ H ├─░───┼───░─┤ I ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░ ┌─┴─┐ ░ ├───┤
q_2: ┤ H ├─░───■───░─┤ I ├─░─┤ X ├─░─┤ H ├
     ├───┤ ░       ░ ├───┤ ░ └───┘ ░ ├───┤
q_3: ┤ H ├─░───────░─┤ Z ├─░───────░─┤ Z ├
     ├───┤ ░       ░ ├───┤ ░       ░ ├───┤
q_4: ┤ H ├─░───────░─┤ H ├─░───────░─┤ H ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘
OrderedDict([(('X', 'X'), 0), (('X', 'Y'), 0), (('X', 'Z'), 0), (('X', 'I'), 0), (('H', 'X'), 0), (('H', 'Y'), 0), (('H', 'Z'), 0), (('H', 'I'), 0), (('Z', 'X'), 0), (('Z', 'Y'), 0), (('Z', 'Z'), 0), (('Z', 'I'), 0), (('I', 'X'), 0), (('I', 'Y'), 0), (('I', 'Z'), 0), (('I', 'I'), 0), (('S', 'X'), 0), (('S', 'Y'), 0), (('S', 'Z'), 0), (('S', 'I'), 0)])
current operation is:  0 ID X
current operation is:  1 S Z
current operation is:  2 H X
current operation is:  3 H X
current operation is:  4 H X
current operation is:  2 CX

odict_values([1, 0, 0, 0, 6, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [86]:
def generate_A(length=5):
    '''This function generates the A matrix by randomly generating each row, checking if it increases the rank and then continuing onwards
    Inputs: numrow - The number of rows in the A matrix.
            numcol - The number of columns in the A matrix.
    Outputs: A - The A matrix.'''

    A = np.zeros((length, length))
    while np.linalg.matrix_rank(A) < A.shape[1]:
        new_row = generate_row_A()
        A_star = np.vstack([A, new_row])
        if np.linalg.matrix_rank(A_star) > np.linalg.matrix_rank(A):
            A = A_star
    return A

print(generate_A(5))


NameError: name 'generate_row_A' is not defined